In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.mixture import GaussianMixture
from scipy.stats import multivariate_normal

In [2]:
# gmm recovery from a synthetic gmm
# make a representative gmm
def make_gmm(n_components, n_features, random_state=0):
    gmm = GaussianMixture(n_components=n_components, random_state=random_state)
    gmm.means_ = np.random.rand(n_components, n_features)
    covariance_array = []
    for i in range(n_components):
        rand_matrix = np.random.rand(n_features, n_features)
        covariance_array.append(np.dot(rand_matrix, rand_matrix.T))

    gmm.covariances_ = np.array(covariance_array)

    precision_array = [np.linalg.pinv(cov) for cov in covariance_array]
    gmm.precisions_ = np.array(precision_array)
    gmm.precisions_cholesky_ = np.array([np.linalg.cholesky(prec) for prec in precision_array])
    
    gmm.weights_ = np.random.rand(n_components)
    gmm.weights_ /= np.sum(gmm.weights_)
    return gmm

In [3]:
n_components = 5
n_features = 20

my_test_gmm = make_gmm(n_components, n_features)
my_test_gmm.sample(1)

(array([[9.96106275, 6.67582311, 6.10926526, 7.96629442, 4.3538099 ,
         6.56523486, 3.44828647, 4.73070603, 4.41565141, 4.81566433,
         5.47509647, 1.9070171 , 6.47961543, 4.29270114, 5.41078108,
         6.54905368, 2.46165833, 3.61818522, 6.70799447, 8.57542518]]),
 array([2]))

In [4]:
# take compressive meaurements
# make 10 random measurement matrices
n_measurements = ((n_features * 5) // 10) # 50% of the features

measurement_matrices = []
for i in range(10):
    measurement_matrices.append(np.random.randn(n_measurements, n_features))

# generate samples from the gmm
n_samples = 1000
samples = my_test_gmm.sample(n_samples)

# take measurements
compressed_measurements = []
sample_measurement_matrix = []
for i in range(n_samples):
    matrix_index = np.random.randint(0, len(measurement_matrices))
    sample_measurement_matrix.append(matrix_index)
    compressed_measurements.append(np.dot(measurement_matrices[matrix_index], samples[0][i]))


noise_std_dev = 0.05
noise_covariance_matrix = np.eye(n_measurements) * (noise_std_dev * noise_std_dev)
compressed_measurements = np.array(compressed_measurements)
compressed_measurements += np.random.normal(0, noise_std_dev, compressed_measurements.shape)
print(compressed_measurements.shape)
print(samples[0].shape)
print(measurement_matrices[0].shape)
print(len(sample_measurement_matrix))

(1000, 10)
(1000, 20)
(10, 20)
1000


In [5]:
# for each of the 10 measurement matrices we have a separate gmm in the y-domain so let us make all of them p(y|z)

def form_y_gmms(x_gmm, measurement_matrices, noise_covariance_matrix):
    gmm_list = []
    noise_inverse = np.linalg.inv(noise_covariance_matrix)
    for i in range(len(measurement_matrices)):
        gmm_sample = GaussianMixture(n_components=n_components, random_state=0)
        gmm_sample.means_ = x_gmm.means_ @ measurement_matrices[i].T
        covariance_array = []

        for j in range(n_components):
            C_ij = np.linalg.pinv(measurement_matrices[i].T @ noise_inverse @ measurement_matrices[i] + x_gmm.precisions_[j])
            temp_matrix = noise_inverse @ measurement_matrices[i] @ C_ij @ measurement_matrices[i].T @ noise_inverse
            covariance_array.append(np.linalg.pinv(noise_inverse - temp_matrix))

        gmm_sample.covariances_ = np.array(covariance_array)
        gmm_sample.weights_ = x_gmm.weights_

        precision_array = [np.linalg.pinv(covariance_array[i]) for i in range(n_components)]
        gmm_sample.precisions_ = np.array(precision_array)
        gmm_sample.precisions_cholesky_ = np.array([np.linalg.cholesky(precision_array[i]) for i in range(n_components)])
        
        gmm_list.append(gmm_sample)
    return gmm_list


In [6]:
def get_log_likelihood(measurement_matrices, compressed_measurements, sample_measurement_matrix, gmm_list):
    log_likelihood = 0
    for i in range(compressed_measurements.shape[0]):
        # print(gmm_list[sample_measurement_matrix[i]].score(compressed_measurements[i].reshape(1, -1)))
        log_likelihood += gmm_list[sample_measurement_matrix[i]].score(compressed_measurements[i].reshape(1, -1))

    return log_likelihood/len(compressed_measurements)

In [7]:
def get_new_weights(compressed_measurements, y_gmms, sample_measurement_matrix):
    # calculate p_ik
    new_weights = []
    for k in range(n_components):
        ssum = 0
        for i in range(n_samples):
            p_ik = y_gmms[sample_measurement_matrix[i]].weights_[k] * multivariate_normal.pdf(compressed_measurements[i], mean=y_gmms[sample_measurement_matrix[i]].means_[k], cov=y_gmms[sample_measurement_matrix[i]].covariances_[k]) / np.exp(y_gmms[sample_measurement_matrix[i]].score(compressed_measurements[i].reshape(1, -1)))
            ssum += p_ik
        new_weights.append(ssum)
    new_weights = np.array(new_weights)
    new_weights /= np.sum(new_weights)
    return new_weights

In [8]:
def get_x_reconstruction(measurement_matrix, noise_covariance_matrix, x_precision_matrix, x_mean, compressed_measurement):
    """
    gets the x reconstruction for a compressed sample for the z^th component in x (specified by x_covariance_matrix and x_mean)
    """
    noise_inverse = np.linalg.pinv(noise_covariance_matrix)
    C_z = np.linalg.pinv(measurement_matrix.T @ noise_inverse @ measurement_matrix + x_precision_matrix)
    x_reconstruction = x_mean + C_z @ measurement_matrix.T @ noise_inverse @ (compressed_measurement - measurement_matrix @ x_mean)
    return x_reconstruction    

In [9]:
def get_new_means(compressed_measurements, y_gmms, sample_measurement_matrix, estimated_gmm, measurement_matrices, noise_covariance_matrix):
    new_means = []
    for k in range(n_components):
        ssum = 0
        psum = 0
        for i in range(n_samples):
            p_ik = y_gmms[sample_measurement_matrix[i]].weights_[k] * multivariate_normal.pdf(compressed_measurements[i], mean=y_gmms[sample_measurement_matrix[i]].means_[k], cov=y_gmms[sample_measurement_matrix[i]].covariances_[k]) / np.exp(y_gmms[sample_measurement_matrix[i]].score(compressed_measurements[i].reshape(1, -1)))
            psum += p_ik
            n_ik = get_x_reconstruction(measurement_matrices[sample_measurement_matrix[i]], noise_covariance_matrix, estimated_gmm.precisions_[k], estimated_gmm.means_[k], compressed_measurements[i])
            ssum += p_ik * n_ik
        new_means.append(ssum/psum)
    new_means = np.array(new_means)
    return new_means

In [10]:
def get_new_covariances(compressed_measurements, y_gmms, sample_measurement_matrix, estimated_gmm, measurement_matrices, noise_covariance_matrix, new_means):
    new_covariances = []
    for k in range(n_components):
        csum = 0
        psum = 0
        for i in range(n_samples):
            p_ik = y_gmms[sample_measurement_matrix[i]].weights_[k] * multivariate_normal.pdf(compressed_measurements[i], mean=y_gmms[sample_measurement_matrix[i]].means_[k], cov=y_gmms[sample_measurement_matrix[i]].covariances_[k]) / np.exp(y_gmms[sample_measurement_matrix[i]].score(compressed_measurements[i].reshape(1, -1)))
            psum += p_ik
            n_ik = get_x_reconstruction(measurement_matrices[sample_measurement_matrix[i]], noise_covariance_matrix, estimated_gmm.precisions_[k], estimated_gmm.means_[k], compressed_measurements[i])
            C_ik = np.linalg.pinv(measurement_matrices[sample_measurement_matrix[i]].T @ np.linalg.pinv(noise_covariance_matrix) @ measurement_matrices[sample_measurement_matrix[i]] + estimated_gmm.precisions_[k])
            csum += p_ik * (C_ik + (n_ik - new_means[k]).reshape(-1, 1) @ (n_ik - new_means[k]).reshape(1, -1))
        new_covariances.append(csum/psum)
    new_covariances = np.array(new_covariances)
    return new_covariances

In [12]:
# initialise estimate of the original gmm
# assumption is that we know the number of components required to model the original gmm
estimated_gmm = make_gmm(n_components, n_features, random_state=7)

# perform updates on the estimated gmm
# log likelihoods stores the log likelihoods of the estimated gmm computed on the samples drawn from the original gmm, our objective is to maximise this
tol = 0.05
log_likelihoods = []
y_gmms = form_y_gmms(estimated_gmm, measurement_matrices, noise_covariance_matrix)
log_likelihoods.append(get_log_likelihood(measurement_matrices, compressed_measurements, sample_measurement_matrix, y_gmms))

print("Initial log likelihood: ", log_likelihoods[-1])
ideal_log_likelihood = get_log_likelihood(measurement_matrices, compressed_measurements, sample_measurement_matrix, form_y_gmms(my_test_gmm, measurement_matrices, noise_covariance_matrix))
print("Ideal log likelihood: ", ideal_log_likelihood)

noise_inverse =  np.linalg.inv(noise_covariance_matrix)
iter_counter = 0

while True:
    # perform updates
    # for each of the 10 measurement matrices we have a separate gmm in the y-domain so let us make all of them
    # update weights
    # new_weights = []
    # for i in range(n_components):
    #     ssum = 0
    #     for j in range(len(compressed_measurements)):
    #         den = 0
    #         for k in range(n_components):
    #             den += estimated_gmm.weights_[k] * multivariate_normal.pdf(compressed_measurements[j], mean=y_gmms[sample_measurement_matrix[j]].means_[k], cov=y_gmms[sample_measurement_matrix[j]].covariances_[k])
    #         ssum += estimated_gmm.weights_[i] * multivariate_normal.pdf(compressed_measurements[j], mean=y_gmms[sample_measurement_matrix[j]].means_[i], cov=y_gmms[sample_measurement_matrix[j]].covariances_[i]) / den
    #     new_weights.append(ssum) # entry appended was \sum_{i = 1}^{N} p_ik for k as the iter counter (here 'i')
    # new_weights = np.array(new_weights)
    # # new_weights /= np.sum(new_weights) # do this after mean and covariance update

    # # update means
    # new_means = []
    # for i in range(n_components):
    #     ssum = 0
    #     psum = 0
    #     for j in range(len(compressed_measurements)):
    #         den = 0
    #         for k in range(n_components):
    #             den += estimated_gmm.weights_[k] * multivariate_normal.pdf(compressed_measurements[j], mean=y_gmms[sample_measurement_matrix[j]].means_[k], cov=y_gmms[sample_measurement_matrix[j]].covariances_[k])
    #         p_ik = estimated_gmm.weights_[i] * multivariate_normal.pdf(compressed_measurements[j], mean=y_gmms[sample_measurement_matrix[j]].means_[i], cov=y_gmms[sample_measurement_matrix[j]].covariances_[i]) / den
    #         C_ik = np.linalg.pinv(measurement_matrices[sample_measurement_matrix[j]].T @ noise_inverse @ measurement_matrices[sample_measurement_matrix[j]] + estimated_gmm.precisions_[i])
    #         n_ik = estimated_gmm.means_[i] + (C_ik @ measurement_matrices[sample_measurement_matrix[j]].T @ noise_inverse @ (compressed_measurements[j] - measurement_matrices[sample_measurement_matrix[j]] @ estimated_gmm.means_[i]))
    #         ssum += p_ik * n_ik
    #         psum += p_ik
    #     new_means.append(ssum / psum)

    # # update covariances
    # new_covariances = []
    # for i in range(n_components):
    #     ssum = 0
    #     psum = 0
    #     for j in range(len(compressed_measurements)):
    #         den = 0
    #         for k in range(n_components):
    #             den += y_gmms[sample_measurement_matrix[j]].weights_[k] * multivariate_normal.pdf(compressed_measurements[j], mean=y_gmms[sample_measurement_matrix[j]].means_[k], cov=y_gmms[sample_measurement_matrix[j]].covariances_[k])
    #         p_ik = y_gmms[sample_measurement_matrix[j]].weights_[i] * multivariate_normal.pdf(compressed_measurements[j], mean=y_gmms[sample_measurement_matrix[j]].means_[i], cov=y_gmms[sample_measurement_matrix[j]].covariances_[i]) / den
    #         C_ik = np.linalg.pinv((measurement_matrices[sample_measurement_matrix[j]].T @ noise_inverse @ measurement_matrices[sample_measurement_matrix[j]]) + estimated_gmm.precisions_[i])
    #         n_ik = estimated_gmm.means_[i] + C_ik @ (measurement_matrices[sample_measurement_matrix[j]].T @ noise_inverse @ (compressed_measurements[j] - measurement_matrices[sample_measurement_matrix[j]] @ estimated_gmm.means_[i]))
    #         ssum += p_ik * ((n_ik - new_means[i]) @ (n_ik - new_means[i]).T + C_ik)
    #         psum += p_ik
    #     new_covariances.append(ssum / psum)


    estimated_gmm.weights_ = get_new_weights(compressed_measurements, y_gmms, sample_measurement_matrix)
    estimated_gmm.means_ = get_new_means(compressed_measurements, y_gmms, sample_measurement_matrix, estimated_gmm, measurement_matrices, noise_covariance_matrix)
    estimated_gmm.covariances_ = get_new_covariances(compressed_measurements, y_gmms, sample_measurement_matrix, estimated_gmm, measurement_matrices, noise_covariance_matrix, estimated_gmm.means_)
    precision_array = [np.linalg.pinv(cov) for cov in estimated_gmm.covariances_]
    estimated_gmm.precisions_ = np.array(precision_array)
    estimated_gmm.precisions_cholesky_ = np.array([np.linalg.cholesky(prec) for prec in estimated_gmm.precisions_])

    y_gmms = form_y_gmms(estimated_gmm, measurement_matrices, noise_covariance_matrix)
    log_likelihoods.append(get_log_likelihood(measurement_matrices, compressed_measurements, sample_measurement_matrix, y_gmms))
    print(f"Iteration {iter_counter + 1} log-likelihood: {log_likelihoods[-1]}")
    iter_counter += 1
    if abs(log_likelihoods[-1] - log_likelihoods[-2]) < tol or log_likelihoods[-1] > ideal_log_likelihood:
        if abs(log_likelihoods[-1] - log_likelihoods[-2]) < tol:
            print("Converged")
        else:
            print("Ideal log-likelihood reached, stopping to avoid overfitting")
        break


Initial log likelihood:  -32.89812176828909
Ideal log likelihood:  -30.84889044035522
Iteration 1 log-likelihood: -31.84707499720047
Iteration 2 log-likelihood: -31.57610682324713
Iteration 3 log-likelihood: -31.431651140753964
Iteration 4 log-likelihood: -31.334034693266776
Iteration 5 log-likelihood: -31.261775000544326
Iteration 6 log-likelihood: -31.20477622645312
Iteration 7 log-likelihood: -31.156885467948406
Converged
